<center><img src='https://github.com/MateusPincho/Simulacao-SEP/blob/master/docs/logo.png?raw=true'/></center>

<h1 align = 'center'>Trabalho de Circuitos Elétricos II<h1/>
<h2 align = 'center'>Modelagem de Sistema Elétrico de Potência<h2/>

Este projeto tem como objetivo o estudo referente ao dimensionamento de um Filtro Ativo. O projeto está salvo e disponível para consulta em nosso [repositório](https://github.com/MateusPincho/Simulacao-SEP/tree/master).

Projetistas:

- André Vitor Cadena de Oliveira
- Fábio Augusto Almeida Marçal
- Gabriel Nazário Gonçalves
- Lorenzo Carrera de Oliveira
- Mateus Pincho de Oliveira

---
 
Segundo a teoria de quadripolos, podemos modelar os componentes do sistema elétrico de potência da seguinte forma: 

### Linha de Transmissão

<center><img src='https://github.com/MateusPincho/Simulacao-SEP/blob/master/docs/linha.png?raw=true'/></center>

Linhas de transmissão possuem as suas características distríbuidas por toda linha, sendo a impedância $Z = R + jX_L$ e sua admitância $Y = jX_C$. Para linhas de transmissão entre 80 a 250 quilômetros, podemos concentrar os seus parâmetros numa equação matricial. A tensão e corrente no começo e final da linha de transmissão podem ser determinadas como:

$$ \begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 + \frac{ZY}{2} & Z\\ 
Y+\frac{ZY^{2}}{4} & 1 + \frac{ZY}{2}
\end{bmatrix} \cdot \begin{bmatrix}
V_2\\ 
-I_2
\end{bmatrix} $$

---

### Transformador

<center><img src='https://github.com/MateusPincho/Simulacao-SEP/blob/master/docs/transformador.png?raw=true'/></center>

Um transformador de um sistema elétrico de potência pode ser modelador como mostrado na figura anterior. A relação de tensão e corrente nos seus terminais pode ser escrita como: 

$$
\begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
\frac{1}{n}(1+YZ_1) &n(Z_1+Z_2+YZ_1Z_2) \\ 
\frac{1}{n}Y & n(1+YZ_2)
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
-I_2
\end{bmatrix}
$$

Sendo: 

$$Z_1 = R_1+jX_{L_1}$$
$$Z_2 = R_2+jX_{L_2}$$

$$Y = \frac{R_m + jX_m}{R_m \cdot jX_m}$$

---

### Carga em Série

A tensão e corrente nos terminais de uma carga colocada em série com o circuito é dada por:

$$\begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 &Z \\ 
0 & 1
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
-I_2
\end{bmatrix} $$

---

### Carga em Paralelo

A tensão e corrente nos terminais de uma carga colocada em paralelo com o circuito é dada por:

$$ \begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 &0 \\ 
Y & 1
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
-I_2
\end{bmatrix}$$

---

In [751]:
# Importando os módulos utilizados...
import numpy as np

In [752]:
# Classes para Quadripolos de Sistemas de Transmissão

class Quadripolo:
    def __init__(self, T=np.zeros((2,2))):
        self.T = T  
        self.A = self.T[0][0]
        self.B = self.T[0][1]
        self.C = self.T[1][0]
        self.D = self.T[1][1]

    @staticmethod
    def paralelo(QDP1, QDP2):
        A = (QDP1.A*QDP2.B+QDP2.A*QDP1.B)/(QDP1.B+QDP2.B) 
        B = (QDP1.B*QDP2.B)/(QDP1.B+QDP2.B)             
        C = QDP1.C+QDP2.C+(QDP1.A-QDP2.A)*(QDP2.D-QDP1.D)/(QDP1.B+QDP2.B)
        D = (QDP2.B*QDP1.D+QDP1.B*QDP2.D)/(QDP1.B+QDP2.B)

        T = np.array([[A, B], 
                      [C, D]])
        
        return Quadripolo(T)

class LinhaDeTransmissao(Quadripolo):
    def __init__(self, w, rho_R, rho_L, rho_C, km):

        R, L, C = km * rho_R, km * rho_L, km * rho_C
        Z = complex(R, w*L)
        Y = complex(0, w*C)
        T = np.array([[1+Z*Y/2, Z], 
                      [Y+Z*(Y**2)/4, 1+Z*Y/2]])
        
        super().__init__(T)
        
class Transformador(Quadripolo):
    def __init__(self, n, Z1, Z2, Ym):
        T = np.array([[(1/n)*(1+Ym*Z1), n*(Z1+Z2+Ym*Z1*Z2)], 
                      [(1/n)*Ym, n*(1+Ym*Z2)]])
        
        super().__init__(T)
        
class CargaEmSerie(Quadripolo):
    def __init__(self, Z):
        T = np.array([[1, Z],
                      [0, 1]])  
        
        super().__init__(T)

class CargaEmDerivacao(Quadripolo):
    def __init__(self, Z):
        T = np.array([[1, 0],
                      [1/Z, 1]])  
        
        super().__init__(T)

In [753]:
# Classe para um Sistema Elétrico de Potência

class SEP: 
    def __init__(self, f):
        self.f = f            # Frequência de operação do sistema em Hz
        self.w = 2*np.pi*f    # Frequência de operação do sistema em rad/s
        self.STC = []         # Sequência de Transmissão em Cascata (geração -> distribuição)

    # Retorna o quadripolo total que representa o Sistema Elétrico de Potência do elemento i ao elemento f
    # Para retornar a transformação do terminal 1 para o 2, basta por inv como True
    def QDP(self, i=1, f=None, inv=False):
        T = self.STC[i-1]

        for t in self.STC[i:f]:
            T = T @ t

        if inv: # T será sua matriz de transmissão inversa
            T = np.linalg.inv(T) @ np.array([[1,0],
                                             [0,-1]])

        return Quadripolo(T)

    # Adiciona um quadripolo em cascata com o último quadripolo adicionado
    # Se não há nenhum quadripolo adicionado, adiciona em cascata com a fonte geradora
    def cascata(self, QDP): 
        self.STC.append(QDP.T)

    # Edita o elemento da posição p
    def edit(self, QDP, p):
        self.STC[p-1] = QDP.T

In [754]:
# Funções auxiliares para a resolução

# Retorna a notação fasorial de um vetor complexo
def fasor(v):
    return f'{np.abs(v):.2f}∠{np.angle(v, deg=True):.1f}°' 

## Questão 1

<center><img src='https://github.com/MateusPincho/Simulacao-SEP/blob/master/docs/SEP.png?raw=true'/></center>

Considerando o sistema elétrico acima operando a $60 \text{ Hz}$, modele este sistema criando as matrizes de transferência $[T]$ para cada elemento a partir dos dados de entrada do modelo:

**Impedância em série de Thévenin** 
- $R_f = 4\Omega$;
- $X_f = 0,38\Omega$. 
  
**Parâmetros das Linhas de Transmissão**
- $\rho_R = 0,172 \Omega\text{/km}$;
- $\rho_L = 2,18 \text{mH/km}$;
- $\rho_C = 0,0136 \mu\text{F/km}$.
  
Considere que ${LT}_1, {LT}_2$ e ${LT}_3$ possuem $80 \text{ km}$ e que ${LT}_4$ possui $120 \text{ km}$.

**Parâmetros comuns dos Transformadores**
- $R_1 = 7,6 \text{ m}\Omega$;
- $X_1 = 3,8 \text{ m}\Omega$;
- $R_2 = 33,0 \text{ m}\Omega$;
- $X_2 = 0,85 \text{ m}\Omega$.

**Parâmetros individuais dos Transformadores**
- Para o Transformador 1:
  - $R_m = 4320\Omega$ e $X_m = 5050\Omega$;
- Para o Transformador 2:
  - $R_m = 432000\Omega$ e $X_m = 505000\Omega$;
- Para o Transformador 3:
  - Rm = $R_m = 402000\Omega$ e $X_m = 607000\Omega$.

**Cargas em Derivação**
- $R_1 = 8000\Omega$ e $L_1 = 41\text{H}$;
- $R_2 = 1350,55\Omega$ e $L_2 = 7,83\text{H}$;
- $R_3 = 649\Omega$ e $L_3 = 3,2\text{H}$.

---

In [755]:
# Implementando o SEP em Python

_SEP = SEP(60) # SEP operando a 60 Hz

# Impedância de Thévenin da Fonte
Rf, Xf = 4, 0.38j
Zf = Rf + Xf 
_SEP.cascata(CargaEmSerie(Zf))

# Parâmetros das impedâncias de entrada e saída dos transformadores

Rt1, Xt1 = 7.6e-3, 3.8e-3j
Zt1 = Rt1 + Xt1 # Impedância de entrada

Rt2, Xt2 = 33.9e-3, 0.85e-3j
Zt2 = Rt2 + Xt2 # Impedância de saída

# Transformador 1

Rm1, Xm1 = 4.32e3, 5.05e3j 
Ym1 = 1/Rm1 + 1/Xm1 # Admitância interna do transformador 1
n1 = 500/69 # TAP do transformador 1

_SEP.cascata(Transformador(n1, Zt1, Zt2, Ym1))

# Linhas em Paralelo 1 e 2

rho_R = 0.172     # Ohm/km
rho_L = 2.18e-3   # Henry/km
rho_C = 0.0136e-6 # Farad/km

LT1 = LinhaDeTransmissao(_SEP.w, rho_R, rho_L, rho_C, 80)
LT2 = LinhaDeTransmissao(_SEP.w, rho_R, rho_L, rho_C, 80)

_SEP.cascata(Quadripolo.paralelo(LT1, LT2))

# Carga em Derivação 1

R1, L1 = 8000, 41 
Z1 = complex(R1,(_SEP.w*L1)) 

_SEP.cascata(CargaEmDerivacao(Z1))

# Linha De Transmissão 3

_SEP.cascata(LinhaDeTransmissao(_SEP.w, rho_R, rho_L, rho_C, 80))

# Transformador 2

Rm2, Xm2 = 4.32e5, 5.05e5j 
Ym2 = 1/Rm2 + 1/Xm2 # Admitância interna do transformador 2
n2 = 230/500

_SEP.cascata(Transformador(n2, Zt1, Zt2, Ym2))

# Carga em Derivação 2

R2, L2 = 1350.55, 7.83 
Z2 = complex(R2,(_SEP.w*L2)) 

_SEP.cascata(CargaEmDerivacao(Z2))

# Linha De Transmissão 4

_SEP.cascata(LinhaDeTransmissao(_SEP.w, rho_R, rho_L, rho_C, 120))

# Transformador 3

Rm3, Xm3 = 4.02e5, 6.07e5j 
Ym3 = 1/Rm3 + 1/Xm3 # Admitância interna do transformador 3
n3 = 69/230

_SEP.cascata(Transformador(n3, Zt1, Zt2, Ym3))

# Carga em Derivação 3

R3, L3 = 649, 3.2 
Z3 = complex(R3,(_SEP.w*L3))

_SEP.cascata(CargaEmDerivacao(Z3))

## Montagem no Ambiente Virtual - *LTspice*

Utilizando o modelo apresentado anteriormente para os elementos do sistema elétrico de potência, utilizou-se o software *LTspice* para simular o sistema e comparar os resultados encontrados na modelagem em *Python*.

<center><img src='https://github.com/MateusPincho/Simulacao-SEP/blob/master/simulation/SEP-SCH.png?raw=true'/></center>

Vale ressaltar que para simular o modelo de quadripolos do transformador, foram utilizadas fontes de tensão e corrente dependentes para agir como um transformador ideal que fica no estágio final do quadripolo. Essa mudança foi feita pois o *LTspice* não possui um elemento que simule um transformador ideal.

<center><img src='https://github.com/MateusPincho/Simulacao-SEP/blob/master/simulation/Tideal.png?raw=true'/></center>

Com essa implementação, a modelagem do sistema por quadripolos apresentou resultado similar a análise por elementos de circuitos simulados no *LTspice*.

---

### Item 1

Obtenha a tensão fasorial $V_{ac}$ e a corrente fasorial $I_{ac}$ no gerador necessários para que a tensão na carga $Z_3$ seja $69\text{kV}_{RMS}$.

---

In [756]:
# Na Carga Z3

V3 = 69e3 # Assumindo V3 como 69kV RMS
I3 = V3 / Z3

S3 = np.array([[V3], 
               [-I3]])

# No Gerador

Eac = _SEP.QDP().T @ S3

[[Vac],
 [Iac]] = Eac

print('\tTensão e Corrrente Fasoriais no Gerador: ')
print(f'\t• Vac = {fasor(Vac)}')
print(f'\t• Iac = {fasor(Iac)}')

	Tensão e Corrrente Fasoriais no Gerador: 
	• Vac = 65618.83∠16.5°
	• Iac = 4285.43∠87.9°


Rodando a simulação dos instantes $100 \text{ ms}$ até o instante $150 \text{ ms}$ para desconsiderar o transiente inicial, é possível confirmar que, ao aplicar $V_{ac} = 66,61\text{kV}_{RMS}$ como fonte geradora, a tensão na carga $Z_3$ se aproxima de $V_3 = 69\text{kV}_{RMS}$, e a corrente na fonte geradora também se aproxima de $4,28 \text{kA}_{RMS}$, como mostra a figura abaixo:

<center><img src='https://github.com/MateusPincho/Simulacao-SEP/blob/master/simulation/Eac.png?raw=true'/></center>

---

### Item 2

Obtenha a tensão e a corrente fasoriais nas cargas $Z_1$ e $Z_2$.

---

In [757]:
# Por meio do quadripolo equivalente do último elemento até o elemento na saída de Z1

E1 = _SEP.QDP(i=4, f=10).T @ S3

[[V1],
 [_]] = E1

I1 = V1/Z1

print('\tTensão e Corrrente Fasoriais na Carga Z1: ')
print(f'\t• V1 = {fasor(V1)}')
print(f'\t• I1 = {fasor(I1)}\n')

# Por meio do quadripolo equivalente do último elemento até o elemento na saída de Z2

E2 = (_SEP.QDP(i=7, f=10).T @ S3)

[[V2],
 [_]] = E2

I2 = V2/Z2

print('\tTensão e Corrrente Fasoriais na Carga Z2: ')
print(f'\t• V2 = {fasor(V2)}')
print(f'\t• I2 = {fasor(I2)}')


	Tensão e Corrrente Fasoriais na Carga Z1: 
	• V1 = 476371.19∠0.7°
	• I1 = 27.37∠-61.9°

	Tensão e Corrrente Fasoriais na Carga Z2: 
	• V2 = 223076.50∠0.4°
	• I2 = 68.72∠-65.0°


Com as mesmas configurações de simulação, também é possível reconfirmar os valores de tensão e corrente fasoriais das cargas $Z_1$ e $Z_2$ pela simulação.

Para a Carga em Derivação $Z_1$:

<center><img src='https://github.com/MateusPincho/Simulacao-SEP/blob/master/simulation/Z1.png?raw=true'/></center>

Para a Carga em Derivação $Z_2$:

<center><img src='https://github.com/MateusPincho/Simulacao-SEP/blob/master/simulation/Z2.png?raw=true'/></center>

---

### Item 3

Determine o ajuste do TAP $(n)$ dos transformadores $T_1$, $T_2$ e $T_3$ para que as tensões nas cargas sejam, respectivamente, $500\text{kV}_{RMS}$, $230\text{kV}_{RMS}$ e $69\text{kV}_{RMS}$.

---

In [758]:
# Entrada do Gerador

Eac = np.array([[Vac],
                [-Iac]])

# Ajuste do Transformador 1

V1_alvo = 500e3
V1 = (_SEP.QDP(i=1, f=3, inv=True).T @ Eac)[0][0]

print('\tAntes do Ajuste de TAP:')
print(f'\t• V1 = {fasor(V1)}')

# Ajusta o TAP com uma tolerância de 0.1% para a tensão desejado
while abs(V1_alvo - abs(V1)) > 0.001 * V1_alvo:

    # Aumenta o TAP para chegar no alvo
    if V1_alvo > abs(V1):
        n1 += 0.001
    # Diminui o TAP para chegar no alvo
    if V1_alvo < abs(V1):
        n1 -= 0.001

    _SEP.edit(Transformador(n1, Zt1, Zt2, Ym1), 2)

    # Recalcula a tensão obtida
    V1 = (_SEP.QDP(i=1, f=3, inv=True).T @ Eac)[0][0]

print('\tApós do Ajuste de TAP:')
print(f'\t• V1 = {fasor(V1)}')
print(f'\t• Tap ajustado: {n1:.3f}\n')

# Ajuste do Transformador 2

V2_alvo = 230e3
V2 = (_SEP.QDP(i=1, f=6, inv=True).T @ Eac)[0][0]

print('\tAntes do Ajuste de TAP:')
print(f'\t• V2 = {fasor(V2)}')

# Ajusta o TAP com uma tolerância de 0.1% para a tensão desejado
while abs(V2_alvo - abs(V2)) > 0.001 * V2_alvo:

    # Aumenta o TAP para chegar no alvo
    if V2_alvo > abs(V2):
        n2 += 0.001
    # Diminui o TAP para chegar no alvo
    if V2_alvo < abs(V2):
        n2 -= 0.001

    _SEP.edit(Transformador(n2, Zt1, Zt2, Ym1), 6)

    # Recalcula a tensão obtida
    V2 = (_SEP.QDP(i=1, f=6, inv=True).T @ Eac)[0][0]

print('\tApós do Ajuste de TAP:')
print(f'\t• V2 = {fasor(V2)}')
print(f'\t• Tap ajustado: {n2:.3f}\n')

# Ajuste do Transformador 3

V3_alvo = 69e3
V3 = (_SEP.QDP(i=1, f=9, inv=True).T @ Eac)[0][0]

print('\tAntes do Ajuste de TAP:')
print(f'\t• V3 = {fasor(V3)}')

# Ajusta o TAP com uma tolerância de 0.1% para a tensão desejado
while abs(V3_alvo - abs(V3)) > 0.001 * V3_alvo:

    # Aumenta o TAP para chegar no alvo
    if V3_alvo > abs(V3):
        n3 += 0.0001
    # Diminui o TAP para chegar no alvo
    if V3_alvo < abs(V3):
        n3 -= 0.0001

    _SEP.edit(Transformador(n3, Zt1, Zt2, Ym1), 9)

    # Recalcula a tensão obtida
    V3 = (_SEP.QDP(i=1, f=9, inv=True).T @ Eac)[0][0]

print('\tApós do Ajuste de TAP:')
print(f'\t• V3 = {fasor(V3)}')
print(f'\t• Tap ajustado: {n3:.3f}\n')


	Antes do Ajuste de TAP:
	• V1 = 476371.19∠0.7°
	Após do Ajuste de TAP:
	• V1 = 499500.77∠0.8°
	• Tap ajustado: 7.628

	Antes do Ajuste de TAP:
	• V2 = 232127.74∠0.6°
	Após do Ajuste de TAP:
	• V2 = 230111.06∠0.6°
	• Tap ajustado: 0.456

	Antes do Ajuste de TAP:
	• V3 = 75732.56∠5.5°
	Após do Ajuste de TAP:
	• V3 = 69068.65∠5.5°
	• Tap ajustado: 0.274



Ajustando o TAP dos 3 Transformadores para o calculado pelo código, os valores de $V_1, \space V_2, \space V_3$ são avaliados pela simulação:

<center><img src='https://github.com/MateusPincho/Simulacao-SEP/blob/master/simulation/T.png?raw=true'/></center>

Como os valores estão satisfatoriamente próximos aos desejados, conclui-se que o ajuste de TAP foi adequado.

---